# 마르코프 체인
![](https://jeremykun.files.wordpress.com/2015/02/markov01.gif?w=1800)


* 이전 상태가 다음 상태의 영향을 주는것으로 1개 또는 N개의 이전 상태가 주어졌을 때 다음상태가 나올 확률을 계산하여 실절적인 의미의 연관성을 생각하지 않고 문장을 조합.
* 조금 이상한 문장이 만들어지는 경우가 있음. 

## 문장을 만드는 과정
1. 문장을 단어로 분할
2. 단어의 전후 연결을 딕셔너리에 등록
3. 사전을 사용해 임의의 문장을 생성 

## 예제 
* 각 문장의 형태소 분석 
 * 개|도|닷새|가|되면|주인|을|안다.
 * 기르던|개|에게|다리|가|물렸다.
 * 닭|쫒던|개|지붕|쳐다|보듯|한다.
 * 똥|묻은|개|가|겨|묻은|개|나무란다
 
* 단어의 전후 딕셔너리 등록 
 * 개 : 가,에게,지붕,도
 * 가 : 되면,겨
 * 되면 : 주인
 * 주인 : 주인을 안다. 
 
* 문장생성
 * "개가 되면 주인을 안다."



In [3]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request
import os, json, random


# 마르코프 체인 딕셔너리 만들기 --- (※1)
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic


# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3
    
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
        
    dic[w1][w2][w3] += 1


In [4]:
# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic"
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".": break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 결과물 
    #'희번덕이는섬진강저켠은전라도땅이켠은경상도땅너그럽게그어진능선은확실한윤곽을드러낸다.'
    # 띄어쓰기
    params = urllib.parse.urlencode({"_callback": "", "q": ret})
    # 네이버 맞춤법 검사기를 사용합니다.
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/dcontent/spellchecker.nhn?" + params)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return data


def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))


In [7]:
# 문장 읽어 들이기 --- (※4)
dict_file = "markov-toji_kk.json"
if not os.path.exists(dict_file):
    # 토지 텍스트 파일 읽어 들이기
    fp = codecs.open("BEXX0003.txt", "r", encoding="utf-16")
    soup = BeautifulSoup(fp, "html.parser")
    body = soup.select_one("body > text")
    text = body.getText()
    text = text.replace("…", "")  # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    # 형태소 분석
    twitter = Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file, "w", encoding='utf-8'), ensure_ascii=False)
else:
    dic = json.load(open(dict_file, "r"))

# 문장 만들기 --- (※6)
for i in range(10):
    s = make_sentence(dic)
    print(s)
    print("---")

외딴 오두막으로 들어간다.
---
그렇게 무서워하던 할머니였는데도.
---
쭈빗쭈빗한논둑의마른가장귀같았다.
---
발가벗은 나무들은 다시 주막으로 들어간 귀녀는 순간 막연해지는 모양이다.
---
남자들이란다 바람기는 있기 마련인데 조만간에 이동진이처음으로 모든 것이 식구들은 물밑을 헤엄치는고기떼 모양 조용히 움직이고 있었다 할 수 없제.
---
그럼그렇지가업어다디리겄십니다마는머어이러다가짚불 잦아지고 소나아 간장을 녹이는 기이머 있노 삼수 너도 마시느라 술을 못하게 방해를 한번 맺은부부의 인연은 풀 수 없는 강청댁으로서는 장날이야말로 원수일밖에 없다 그 말 아닙니까.
---
내외간의 정분이 없는 가배 이야기가 빗나간 것은 아니니까하듯이 건달풍의 사내가 못할 개인데 이걸 알면 겁이 났다.
---
뭐라고 말을 끝맺을 자식 네 땜에 그럴 것이라고 생각했었다.
---
적반하장도 푼수에 넘는 돌다리가 걸려있었다 할 수 있겄십니까.
---
쇠돌이 가 딱해하여 거들었다.
---


# 문장 만들어지는 과정

* 형태소 분석 
***
"제1편 어둠의 발소리"
***
* 딕셔너리 등록 
***
{'@': {'제': {'1': 1}}, '제': {'1': {'편': 1}}, '1': {'편': {'어둠': 1}}, '편': {'어둠': {'의': 1}}, '어둠': {'의': {'발': 1}}, '의': {'발': {'소리': 1}}, '발': {'소리': {'서': 1}}, '소리': {'서': {'序': 1}}}...
***
* 문장 생성
***
렌덤으로 첫문장가져와서 해당 맞는 문장가져오기 
***

# LSTM / RNN
* RNN (Recurrent Neural Network)
 * 신경망을 재귀적으로 사용해 시간순서를 가진 데이터
* LSTM (Long Short Term-Memory)
 * 장기적으로 기억
 
[ 너무 어렵다~ ]